In [14]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch import nn, optim

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers
import io
import re
import string
import tqdm
from tqdm.notebook import tqdm

## Load and Train Word2Vec

In [123]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
    # Elements of each training example are appended to these lists.
    targets, contexts, labels = [], [], []

    # Build the sampling table for `vocab_size` tokens.
    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

    # Iterate over all sequences (sentences) in the dataset.
    for sequence in tqdm(sequences):

        # Generate positive skip-gram pairs for a sequence (sentence).
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
              sequence,
              vocabulary_size=vocab_size,
              sampling_table=sampling_table,
              window_size=window_size,
              negative_samples=0)

        # Iterate over each positive skip-gram pair to produce training examples
        # with a positive context word and negative samples.
        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims(
                  tf.constant([context_word], dtype="int64"), 1)
            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
                  true_classes=context_class,
                  num_true=1,
                  num_sampled=num_ns,
                  unique=True,
                  range_max=vocab_size,
                  seed=seed,
                  name="negative_sampling")

              # Build context and label vectors (for one target word)
            context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
            label = tf.constant([1] + [0]*num_ns, dtype="int64")

            # Append each element from the training example to global lists.
            targets.append(target_word)
            contexts.append(context)
            labels.append(label)

    return targets, contexts, labels


In [115]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [116]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

C:\Users\William\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [117]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [118]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)


In [119]:
vectorize_layer.adapt(text_ds.batch(1024))

In [120]:
inverse_vocab = vectorize_layer.get_vocabulary()
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [121]:

sequences = list(text_vector_ds.as_numpy_iterator())

In [124]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)


  0%|          | 0/32777 [00:00<?, ?it/s]



targets.shape: (65099,)
contexts.shape: (65099, 5)
labels.shape: (65099, 5)
<BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [ ]:
num_ns = 4
class Word2Vec(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.target_embedding = layers.Embedding(vocab_size,
                                          embedding_dim,
                                          input_length=1,
                                          name="w2v_embedding")
        self.context_embedding = layers.Embedding(vocab_size,
                                           embedding_dim,
                                           input_length=num_ns+1)

    def call(self, pair):
        target, context = pair
        # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
        # context: (batch, context)
        if len(target.shape) == 2:
            target = tf.squeeze(target, axis=1)
        # target: (batch,)
        word_emb = self.target_embedding(target)
        # word_emb: (batch, embed)
        context_emb = self.context_embedding(context)
        # context_emb: (batch, context, embed)
        dots = tf.einsum('be,bce->bc', word_emb, context_emb)
        # dots: (batch, context)
        return dots
def custom_loss(x_logit, y_true):
    return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)


In [125]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [126]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [127]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
63/63 [==============================] - 5s 80ms/step - loss: 1.6082 - accuracy: 0.2343
Epoch 2/20
63/63 [==============================] - 5s 78ms/step - loss: 1.5890 - accuracy: 0.5527
Epoch 3/20
63/63 [==============================] - 5s 80ms/step - loss: 1.5417 - accuracy: 0.5894
Epoch 4/20
63/63 [==============================] - 5s 79ms/step - loss: 1.4599 - accuracy: 0.5661
Epoch 5/20
63/63 [==============================] - 5s 79ms/step - loss: 1.3624 - accuracy: 0.5754
Epoch 6/20
63/63 [==============================] - 5s 78ms/step - loss: 1.2653 - accuracy: 0.6050
Epoch 7/20
63/63 [==============================] - 6s 88ms/step - loss: 1.1742 - accuracy: 0.6408
Epoch 8/20
63/63 [==============================] - 5s 83ms/step - loss: 1.0893 - accuracy: 0.6755
Epoch 9/20
63/63 [==============================] - 5s 79ms/step - loss: 1.0103 - accuracy: 0.7095
Epoch 10/20
63/63 [==============================] - 5s 82ms/step - loss: 0.9370 - accuracy: 0.7392
Epoch 11/

## Distance

In [140]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [141]:
embeddings = {}
for idx, word in enumerate(vocab):
    if idx == 0:
        continue
        
    embeddings[word] = weights[idx]

In [186]:
def sim_dis(word1, word2):
    """
    returns cosine similiarity, disimilarity which is given by mean squared error.
    """
    vec1 = embeddings[word1]
    vec2 = embeddings[word2]

    cosine = np.dot(vec1,vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))
    return cosine, np.abs(vec1-vec2).mean()

In [187]:
sim_dis('this', 'he')

(0.09736582, 0.23770557)

In [188]:
sim_dis('this', 'that')

(0.13859883, 0.2217257)